In [1]:
import pandas as pd
df = pd.read_csv('../../top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [5]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [12]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [14]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [17]:
# vectorize!
# Note that for Coursera we use an older way of Qdrant doing the uploads using Records instead of Points
qdrant.upload_records(
    collection_name="top_wines",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [18]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [19]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Bodega Colome Altura Maxima Malbec 2012', 'region': 'Salta, Argentina', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Winemaker Thibaut Delmotte has crafted wines of distinction and international acclaim for Colome. He believes the Malbec from Altura Maxima Vineyard is the embodiment of two extremes - a traditional grape variety from his French origins made from the vineyard that challenges all convention in the modern viticultural world.'} score: 0.6179680028359475
{'name': 'Catena Zapata Adrianna Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 97.0, 'notes': '"The single-vineyard 2004 Malbec Adrianna Vineyard from the Gualtallary district is inky purple with aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. Opulent, full-flavored, yet remarkably light on its feet, this medium to full-bodied Malbec is all about pleasure. It will certainly evolve for a decade but is hard to resist now. It is a fine test of one\'s 

In [20]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [40]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="[{'name': 'Bodega Norton Malbec 2012', 'region': 'Mendoza, Argentina', 'variety': 'Red Wine', 'rating': 95.0, 'notes': 'Bodega Norton is one of the oldest wineries in Mendoza, Argentina. Founded in 1991, it is located in the heart of the Uco Valley, at an altitude of 1,350 meters above sea level. The 2012 Malbec is a blend of grapes from the best vineyards of the winery. The wine is aged in oak barrels for 12 months, which gives it a rich and complex character. The 2012 vintage is considered one of the best in recent years, and this Malbec is no exception. It has a deep ruby color and a nose of red fruits, such as cherries and raspberries, with a hint of vanilla from the oak aging. On the palate, it is full-bodied and smooth, with a long and persistent finish. This Malbec is a great example of the quality and tradition of Bodega Norton. It is a wine that can be enjoyed now, but it will also age well for the next 5-10 years.'}]</s>", role='assistant', func